## Preprocess

In [3]:
# Make sure our working directory is the root of project, e.g /amr-parser
%cd ..
%pwd

c:\Users\bandi\MasterDegree\AMR\amr-parser


'c:\\Users\\bandi\\MasterDegree\\AMR\\amr-parser'

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [6]:
import stanza
import os
import pandas as pd
import string
from wordcloud import WordCloud, STOPWORDS 
from IPython.display import display, HTML

In [7]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True) #kalau cudablas nya ga mau, run ini sebelum import get_entities

In [8]:
import penman
from importlib import reload
reload(penman)

<module 'penman' from 'D:\\miniconda3\\envs\\amr\\lib\\site-packages\\penman.py'>

In [10]:
import numpy as np
import pandas as pd
import stanfordnlp
import nltk

import matplotlib.pyplot as plt

from collections import Counter, defaultdict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from utils.ner.entity_recognizer import get_entities, get_entities_tf

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from stog.utils.params import Params
from stog.data.dataset_builder import dataset_from_params, iterator_from_params
from utils.amr_parsing.io import AMRIO

In [17]:
my_dir = "./"

AMR_FILEPATH = "data/amr_id_v2/train.txt"
AMR_TEST_FILEPATH = "data/raw/amr_simple_test.txt"
EDGE_PREDICTION_DATADIR = 'data/raw/edge_prediction/'

# Dependency feature
DEPENDENCY_FEATURES_FILEPATH = EDGE_PREDICTION_DATADIR + 'dependency_parser_features.csv'
TEST_DEPENDENCY_FEATURES_FILEPATH = EDGE_PREDICTION_DATADIR + 'test_dependency_parser_features.csv'

# Filtered dependency features
FILTERED_DEPENDENCY_FEATURES_FILEPATH = EDGE_PREDICTION_DATADIR + 'dependency_parser_features_filtered.csv'
FILTERED_TEST_DEPENDENCY_FEATURES_FILEPATH = EDGE_PREDICTION_DATADIR + 'test_dependency_parser_features_filtered.csv'

# Labeled
LABELED_TEST_DEPENDENCY_FEATURES_FILEPATH = EDGE_PREDICTION_DATADIR + 'test_dependency_parser_features_labeled.csv'
LABELED_DEPENDENCY_FEATURES_FILEPATH = EDGE_PREDICTION_DATADIR + 'dependency_parser_features_labeled.csv'
LABELED_DEPENDENCY_FEATURES_FILEPATH_NO_UNK = EDGE_PREDICTION_DATADIR + 'dependency_parser_features_labeled_no_unk.csv'

# AMR
AMR_DF_FILEPATH = EDGE_PREDICTION_DATADIR + 'amr.csv'
TEST_AMR_DF_FILEPATH = EDGE_PREDICTION_DATADIR + 'amr.test.csv'

# ANNOTATOR 
ANNOTATOR_PARAMS={
    'pos_tagger': 'nltk'
}

# FILTER TYPE
FILTER_TYPE = ['preposition', 'determiner', 'sc']

# Severina Dataset
TEST_BERITA_DIR = my_dir+"data/raw/test/"
TEST_BERITA_FILE_NAMES = ["b-salah-darat.txt","c-gedung-roboh.txt","d-indo-fuji.txt", "f-bunuh-diri.txt", "g-gempa-dieng.txt"]
TEST_BERITA_FILE_PATHS = [TEST_BERITA_DIR + filename for filename in TEST_BERITA_FILE_NAMES]

## Function Definitions

In [10]:
class FeatureAnnotator:
    word_dict = {}
    
    def __init__(self, params):
#         self.nlp = stanfordnlp.Pipeline(lang="id",use_gpu=False, silent=True)
        self.nlp = stanza.Pipeline(lang="id",use_gpu=True,verbose=False, tokenize_pretokenized=True)
        
        factory = StemmerFactory()
        self.stemmer = factory.create_stemmer()
        self.ner = get_entities
        self.params = params
        
        import nltk
        self.pos_tagger = nltk.tag.CRFTagger()
        self.pos_tagger.set_model_file(my_dir+'pretrained/pos_tagger/all_indo_man_tag_corpus_model.crf.tagger')
    def annotate(self, sentence):
        annotation = defaultdict(list)
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        doc = self.nlp(sentence)
        
        annotation['ner_tags'] = self.ner(sentence)
        
        word_dict = defaultdict(int)
        
        for sent in doc.sentences:
            for idx, word in enumerate(sent.words):
                annotation['tokens'].append(word.text)
                stemmed_word = self.stemmer.stem(word.text)                
                if (annotation['ner_tags'][idx] in ['PER', 'ORG']):
                    stemmed_word = word.text.lower()
                word_dict[stemmed_word] += 1
                annotation['lemmas'].append(stemmed_word+'_{}'.format(word_dict[stemmed_word]))
                annotation['pos_tags'].append(word.upos)
#                 annotation['dependency'].append(dict(relation=word.dependency_relation, head=word.governor))
                annotation['dependency'].append(dict(relation=word.deprel, head=word.head))
    
        if self.params['pos_tagger'] == 'nltk':
            annotation['pos_tags'] = [tag[1] for tag in self.pos_tagger.tag(annotation['tokens'])]
            
        
        return annotation

# Preprocess

### Features

In [11]:
def dump_amr_features(amr, annotation, f):
    amr.tokens = annotation['tokens']
    amr.lemmas = annotation['lemmas']
    amr.pos_tags = annotation['pos_tags']
    amr.ner_tags = annotation['ner_tags']
    AMRIO.dump([amr], f)

def create_dataset(annotation, sentence_id):
    edge_labels_dataset = {
        'sentence_id': [],
        'parent': [], 
        'parent_position': [], 
        'child': [], 
        'child_position': [], 
        'is_root' : [],
        'sequence': [],
        'parent_ner': [],
        'child_ner': [],
        'parent_pos': [],
        'dependency_role': [],
        'child_pos': []
    }

    for idx, dependency in enumerate(annotation['dependency']):
        head = dependency['head']
        if (head != 0):
            head_idx = head-1
            edge_labels_dataset['sentence_id'].append(sentence_id)
            edge_labels_dataset['sequence'].append(idx+1)
            edge_labels_dataset['parent'].append(annotation['lemmas'][head_idx])
            edge_labels_dataset['parent_position'].append(head_idx)
            edge_labels_dataset['child'].append(annotation['lemmas'][idx])
            edge_labels_dataset['child_position'].append(annotation['lemmas'].index(annotation['lemmas'][idx]))
            edge_labels_dataset['is_root'].append(1 if annotation['dependency'][head_idx]['head'] == 0 else 0)
            edge_labels_dataset['parent_ner'].append(annotation['ner_tags'][head_idx])
            edge_labels_dataset['child_ner'].append(annotation['ner_tags'][idx])
            edge_labels_dataset['parent_pos'].append(annotation['pos_tags'][head_idx])
            edge_labels_dataset['child_pos'].append(annotation['pos_tags'][idx])
            edge_labels_dataset['dependency_role'].append(dependency['relation'])
            
    return edge_labels_dataset

def create_dependency_parser_feature_df(filepath, params):
    sentence_dataset = []
    word_dict = defaultdict(int)
    amrs = []
    sentence_ids = []
    all_sentences = ''
    annotator = FeatureAnnotator(params)
    
    with open(filepath + '.features', 'w', encoding='utf-8') as f:
        for i, amr in enumerate(AMRIO.read(filepath), 1):
            if i % 100 == 0:
                print('{} processed.'.format(i))

            for word in nltk.tokenize.word_tokenize(amr.sentence):
                word_dict[word.lower()] += 1
                all_sentences += word.lower() + ' '
            #print('amr.sentence',amr.sentence)
            annotation = annotator.annotate(amr.sentence)
            dump_amr_features(amr, annotation, f)
            sentence_data = create_dataset(annotation, i)
            sentence_dataset.append(sentence_data)
            sentence_ids.append(i)
            amrs.append(amr)

    dataset_dict = {
        'sentence_id': sum([sum([sentence_data['sentence_id'] for sentence_data in sentence_dataset], [])],[]),
        'sequence': sum([sum([sentence_data['sequence'] for sentence_data in sentence_dataset], [])],[]),
        'parent': sum([sum([sentence_data['parent'] for sentence_data in sentence_dataset], [])],[]),
        'parent_position': sum([sum([sentence_data['parent_position'] for sentence_data in sentence_dataset], [])],[]), 
        'child': sum([sum([sentence_data['child'] for sentence_data in sentence_dataset], [])],[]), 
        'child_position': sum([sum([sentence_data['child_position'] for sentence_data in sentence_dataset], [])],[]), 
        'is_root' : sum([sum([sentence_data['is_root'] for sentence_data in sentence_dataset], [])],[]),
        'parent_ner': sum([sum([sentence_data['parent_ner'] for sentence_data in sentence_dataset], [])],[]),
        'child_ner': sum([sum([sentence_data['child_ner'] for sentence_data in sentence_dataset], [])],[]),
        'parent_pos': sum([sum([sentence_data['parent_pos'] for sentence_data in sentence_dataset], [])],[]),
        'dependency_role': sum([sum([sentence_data['dependency_role'] for sentence_data in sentence_dataset], [])],[]),
        'child_pos': sum([sum([sentence_data['child_pos'] for sentence_data in sentence_dataset], [])],[])
    }

    amr_dict = {
        'sentence_id': sentence_ids,
        'amr': [str(amr.graph) for amr in amrs]
    }

    sentence_dataset = pd.DataFrame(dataset_dict)
    amr_df = pd.DataFrame(amr_dict)
    
    return sentence_dataset, amr_df, all_sentences, amrs

def filter_dataset(data, mode='all'):
    if ('preposition' in mode):
        data = data[data['child_pos'] != 'IN']
    
    if ('determiner' in mode):
        data = data[data['dependency_role'] != 'det']
        
    if ('case' in mode):
        data = data[data['dependency_role'] != 'case']
    
    if ('sc' in mode):
        data = data[data['child_pos'] != 'SC']
    
    return data

### Pair Label Generation

In [12]:
def create_pair_labels(train_data):
    pairs = []
    for sentence_id, data in enumerate(train_data):
        tokens =  data.fields['tgt_tokens'].tokens[1:-1]
        head_indices = data.fields['head_indices'].labels
        labels = data.fields['head_tags'].labels
        
        for i, index in enumerate(head_indices):
            if (index != 0):
                parent = tokens[index-1]
                child = tokens[i]
                label = labels[i]
                pairs.append((sentence_id+1, str(parent), str(child),str(label)))
                
    return pairs

def find_label(data, pair_labels):
    for pair_label in pair_labels:
        sentence_id, label_parent, label_child, label = pair_label
        if (data['sentence_id'] == sentence_id and data['parent'] == label_parent and data['child']==label_child):
            return label
    return "unk"
    
def create_labels(pair_labels, sentence_dataset, change_unk_to_mod=False, with_flipped=False):
    found_or_not = []
    labels = []
    unk_pairs = []
    pair_labels_dict = dict()
    not_found_count = 0
    found_count = 0
    
    for idx, data in sentence_dataset.iterrows():
        found = False
        flipped_found = False
        flipped_data = None
        
        if (data['sentence_id'] not in pair_labels_dict):
            #ditambah None untuk menampung label hasil prediksi nantinya (Kalau sampai akhir tetap None, artinya : pair tidak dilabeli)
            pair_labels_dict[data['sentence_id']] = [pair_label + (None,) for pair_label in pair_labels if pair_label[0] == data['sentence_id']]   
        cur_pair_labels = pair_labels_dict[data['sentence_id']]
            
        for i, pair_label in enumerate(pair_labels_dict[data['sentence_id']]):
            _, label_parent, label_child, label, ketemu = pair_label
            if (data['parent'].split('_')[0] == label_parent and data['child'].split('_')[0]==label_child):
                labels.append(label)
                found = True
                pair_labels_dict[data['sentence_id']][i] = _, label_parent, label_child, label, label
                break

            elif with_flipped and (data['parent'].split('_')[0] == label_child and data['child'].split('_')[0]==label_parent):
                flipped_found = True
                print('ADA FLIPPED')
                flipped_data = (data['child'], data['parent'], label)
        
        if flipped_found and not found:
            sentence_dataset.loc[idx,'parent'] = flipped_data[0]
            sentence_dataset.loc[idx,'child'] = flipped_data[1]
            labels.append(flipped_data[2])
        
        if not flipped_found and not found:
            not_found_count += 1
            dependency_role = data['dependency_role'] 
            if (dependency_role in ['nsubj']):
                labels.append("ARG0")
            elif (dependency_role in ['obl', 'obj']):
                labels.append('ARG1')
            elif (dependency_role in ['case'] and data['child'] in ['di', 'ke', 'dari']):
                labels.append('location')
            else:
                if change_unk_to_mod:
                    labels.append("mod")
                else: 
                    labels.append("unk")

            unk_pairs.append((data['parent'], data['child'], dependency_role, data['sentence_id']))
            found_or_not.append(False)
        else:
            found_count += 1
            found_or_not.append(True)   
    
    print("Found pairs: ", found_count)
    print("AMR Pair labels: ", len(pair_labels))
    print("Dependency Parser Pair labels: ", len(sentence_dataset))
    print("Not found: ", not_found_count)
    
    precision = found_count/len(sentence_dataset)
    recall = found_count/len(pair_labels)
    print("Pair Precision: ", precision)
    print("Pair Recall: ", recall)
    print("Pair F1: ", 2*((precision*recall)/(precision+recall)))
    
    return labels, unk_pairs, found_or_not, pair_labels_dict

In [18]:
sentence_dataset, amr_df, all_sentences, amrs = create_dependency_parser_feature_df(AMR_FILEPATH, ANNOTATOR_PARAMS)
test_sentence_dataset, test_amr_df, test_all_sentences, test_amrs = create_dependency_parser_feature_df(AMR_TEST_FILEPATH, ANNOTATOR_PARAMS)

100 processed.
200 processed.
300 processed.
400 processed.
500 processed.
600 processed.
700 processed.
800 processed.
900 processed.
1000 processed.
1100 processed.
1200 processed.
1300 processed.
1400 processed.
1500 processed.
1600 processed.
1700 processed.
1800 processed.
1900 processed.
2000 processed.
2100 processed.
2200 processed.
2300 processed.
(d / dan    :op1 (j / jadi            :ARG0 (h / hendra)            :ARG1 (a / alumni                     :mod (l / lulusan                             :mod (b / baik)))            :location (u / universitas                         :mod (i / islam                                 :location (m / malang)))            :time (t / tahun                     :mod (_ / 2013)))    :op2 (d2 / dagang             :ARG0 (a2 / ayah)             :ARG1 (s / siomay)))
Expected "(" at position 358
ERR_START
['(d / dan', '   :op1 (j / jadi', '           :ARG0 (h / hendra)', '           :ARG1 (a / alumni', '                    :mod (l / lulusan', '      

In [20]:
# labels_no_unk, unk_pairs, found_or_not, pair_labels_dict = create_labels(pair_labels[:],filtered_sentence_dataset[:], change_unk_to_mod=True)

In [22]:
# actual = []
# pred = []
# kelas = []
# for i,j in enumerate(pair_labels_dict):
#     for a,b in enumerate(pair_labels_dict[j]):
#         _, label_parent, label_child, label_actual, label_prediction = b
#         actual.append(str(label_actual))
#         pred.append(str(label_prediction))
#         if str(label_prediction) not in kelas:
#             kelas.append(str(label_prediction))

In [23]:
kelas= ['ARG0','ARG1','location','mod','name', 'time','None']

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(actual, pred, kelas)
cm

In [ ]:
jml_label = {}
rasio_not_found = {}
for i,j in enumerate(cm):
    betul = j[i]
    jadi_none = j[len(j)-1]
    rasio_jadi_none = jadi_none/(betul+jadi_none)
    if np.isnan(rasio_jadi_none):
        rasio_jadi_none = 0
    print(kelas[i],'==>',round(rasio_jadi_none,2))
    rasio_not_found[kelas[i]] = round(rasio_jadi_none,2)
    jml_label[kelas[i]] = betul+jadi_none
print(rasio_not_found)
print(jml_label)

In [ ]:
import matplotlib.pyplot as plt

D = rasio_not_found

plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))

plt.show()

In [ ]:
import matplotlib.pyplot as plt

D = jml_label

plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))

plt.show()

In [ ]:
jml_label_adylan = {'ARG0': 665, 'ARG1': 565, 'location': 189, 'mod': 550, 'name': 242, 'time': 158, 'None': 0}
jml_label_amany = jml_label
rasio_not_found_adylan = {'ARG0': 0.21, 'ARG1': 0.17, 'location': 0.28, 'mod': 0.43, 'name': 0.12, 'time': 0.55, 'None': 0}
rasio_not_found_amany = rasio_not_found
hasil = []
for i in jml_label_amany:
    selisih = (jml_label_amany[i]-jml_label_adylan[i])
    rasio = 0
    if jml_label_adylan[i]!= 0:
        rasio = round(jml_label_amany[i]/jml_label_adylan[i],2)
    
    selisih2 = round(abs(rasio_not_found_amany[i]-rasio_not_found_adylan[i]),2)
    rasio2 = 0
    if rasio_not_found_adylan[i]!= 0:
        rasio2 = round(rasio_not_found_amany[i]/rasio_not_found_adylan[i],2)
    hasil.append([i,selisih,rasio,selisih2,rasio2])
    print(i,selisih,rasio,selisih2,rasio2)

In [ ]:
dict(sorted(hasil.items(), key=lambda item: item[1]))

In [ ]:
dict(sorted(jml_label_adylan.items(), key=lambda item: item[1]))

In [ ]:
dict(sorted(jml_label_amany.items(), key=lambda item: item[1]))

In [ ]:
dict(sorted(rasio_not_found_adylan.items(), key=lambda item: item[1]))

In [ ]:
dict(sorted(rasio_not_found_amany.items(), key=lambda item: item[1]))

In [ ]:
from operator import itemgetter
sorted(hasil, key=itemgetter(1)) #diurutkan berdasarkan kenaikan jumlah label

In [ ]:
pred.count('None')/len(pred)

In [ ]:
print(cm)
import seaborn as sns
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(kelas); ax.yaxis.set_ticklabels(kelas);
ax.set_ylim(sorted(ax.get_xlim(), reverse=True)) #supaya confusion matrix nya tidak ke crop
plt.tight_layout()
plt.show()

In [ ]:
TP = 4969

#FP (False Positive ==> Actual Class adalah Negative)
#Pair hasil prediksi yang seharusnya tidak ditemukan (harusnya :tidak ditemukan, tapi hasil prediksi :ada (found))
FP = len(filtered_sentence_dataset) - TP

#FN (False Negative ==> Actual Class adalah Positive)
#Pair Gold yang seharusnya ditemukan (harusnya :ada (found), tapi hasil prediksi :tidak ditemukan)
FN = len(pair_labels) - TP 

precision = TP/(TP+FP)
recall = TP/(TP+FN)

print("Pair Precision: ", precision)
print("Pair Recall: ", recall)
print("Pair F1: ", 2*((precision*recall)/(precision+recall)))

In [ ]:
#Barusan : Sudah run semua (yang found di beri label. yang not found diberi None)
#TO DO : Hitung berapa yang NONE di pair_labels_dict
hitung = 0
for i,j in enumerate(pair_labels_dict):
    #print(i,pair_labels_dict[j])
    for a,b in enumerate(pair_labels_dict[j]):
        #print(a,b)
        _, label_parent, label_child, label_asli, label_pred = b
        #print(label_asli,label_pred)
        if label_pred == None:
            hitung+=1
print(hitung) #pair_labels (gold AMR) yang gak ada di hasil prediksi

In [ ]:
pair_labels_dict

In [ ]:
# def create_labels(pair_labels, sentence_dataset, change_unk_to_mod=False, with_flipped=False):
#     found_or_not = []
#     labels = []
#     unk_pairs = []
#     pair_labels_dict = dict()
#     not_found_count = 0
#     found_count = 0
    
#     for idx, data in sentence_dataset.iterrows():
#         found = False
#         flipped_found = False
#         flipped_data = None
        
#         if (data['sentence_id'] not in pair_labels_dict):
#             pair_labels_dict[data['sentence_id']] = [pair_label for pair_label in pair_labels if pair_label[0] == data['sentence_id']]    
#         cur_pair_labels = pair_labels_dict[data['sentence_id']]
            
#         for pair_label in cur_pair_labels:
#             _, label_parent, label_child, label = pair_label
#             if (data['parent'].split('_')[0] == label_parent and data['child'].split('_')[0]==label_child):
#                 labels.append(label)
#                 found = True
#                 break

#             elif with_flipped and (data['parent'].split('_')[0] == label_child and data['child'].split('_')[0]==label_parent):
#                 flipped_found = True
#                 flipped_data = (data['child'], data['parent'], label)
# #             found_or_not.append(found)
        
#         if flipped_found and not found:
#             sentence_dataset.loc[idx,'parent'] = flipped_data[0]
#             sentence_dataset.loc[idx,'child'] = flipped_data[1]
#             labels.append(flipped_data[2])
        
#         if not flipped_found and not found:
#             not_found_count += 1
#             dependency_role = data['dependency_role'] 
#             if (dependency_role in ['nsubj']):
#                 labels.append("ARG0")
#             elif (dependency_role in ['obl', 'obj']):
#                 labels.append('ARG1')
#             elif (dependency_role in ['case'] and data['child'] in ['di', 'ke', 'dari']):
#                 labels.append('location')
#             else:
#                 if change_unk_to_mod:
#                     labels.append("mod")
#                 else: 
#                     labels.append("unk")

#             unk_pairs.append((data['parent'], data['child'], dependency_role, data['sentence_id']))
#             found_or_not.append(False)
#         else:
#             found_count += 1
#             found_or_not.append(True)   
    
#     print("Found pairs: ", found_count)
#     print("AMR Pair labels: ", len(pair_labels))
#     print("Dependency Parser Pair labels: ", len(sentence_dataset))
#     print("Not found: ", not_found_count)
    
#     precision = found_count/len(sentence_dataset)
#     recall = found_count/len(pair_labels)
#     print("Pair Precision: ", precision)
#     print("Pair Recall: ", recall)
#     print("Pair F1: ", 2*((precision*recall)/(precision+recall)))
    
#     return labels, unk_pairs, found_or_not

In [28]:
%load_ext autoreload


In [30]:
%autoreload

#### Dev data

### [Checkpoint 1 ] Save Dependency Features & AMR

In [15]:
%load_ext autoreload
%autoreload

In [19]:
sentence_dataset.to_csv(DEPENDENCY_FEATURES_FILEPATH, index=None)
test_sentence_dataset.to_csv(TEST_DEPENDENCY_FEATURES_FILEPATH, index=None)
amr_df.to_csv(AMR_DF_FILEPATH, index=None)
test_amr_df.to_csv(TEST_AMR_DF_FILEPATH, index=None)

### Load Dependency Features

In [20]:
sentence_dataset = pd.read_csv(DEPENDENCY_FEATURES_FILEPATH)
test_sentence_dataset = pd.read_csv(TEST_DEPENDENCY_FEATURES_FILEPATH)

## Filtering

In [21]:
# FILTER_TYPE = []
filtered_sentence_dataset = filter_dataset(sentence_dataset, mode=FILTER_TYPE)
filtered_test_sentence_dataset = filter_dataset(test_sentence_dataset, mode=FILTER_TYPE)

### Save Filtered Dependency Features Data

In [22]:
# Do Filter
filtered_sentence_dataset.to_csv(FILTERED_DEPENDENCY_FEATURES_FILEPATH, index=None)
filtered_test_sentence_dataset.to_csv(FILTERED_TEST_DEPENDENCY_FEATURES_FILEPATH, index=None)

### [Checkpoint 2] Load Filtered Dependency Parser Features dataset

In [23]:
filtered_sentence_dataset = pd.read_csv(FILTERED_DEPENDENCY_FEATURES_FILEPATH)
filtered_test_sentence_dataset = pd.read_csv(FILTERED_TEST_DEPENDENCY_FEATURES_FILEPATH)

## Create Label Features

In [24]:
data_params = dict(
    data_dir=my_dir+"data/raw",
    train_data="amr_simple_amany_v2.txt.features",
    test_data="amr_simple_test.txt.features",
    dev_data="amr_simple_amany_v2.txt.features",
    data_type="AMR"
)

amr_dataset = dataset_from_params(data_params)
amr_data = amr_dataset['train']

0it [00:00, ?it/s][2021-05-11 21:40:01,069 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw\amr_simple_amany_v2.txt.features
173it [00:00, 1529.90it/s]

Building train datasets ...
False


2644it [00:02, 1273.31it/s]
0it [00:00, ?it/s][2021-05-11 21:40:03,144 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw\amr_simple_amany_v2.txt.features
223it [00:00, 2138.10it/s]

Building dev datasets ...
False


2644it [00:01, 1632.31it/s]
0it [00:00, ?it/s][2021-05-11 21:40:04,764 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw\amr_simple_test.txt.features
306it [00:00, 2641.47it/s]

Building test datasets ...
False


#### Create Pair Labels

In [41]:
pair_labels = create_pair_labels(amr_data)
labels_no_unk, unk_pairs, found_or_not,_ = create_labels(pair_labels,filtered_sentence_dataset, change_unk_to_mod=True)
dependency_features_dict = filtered_sentence_dataset.to_dict()
dependency_features_dict['label'] = pd.Series(labels_no_unk)
dependency_features_df = pd.DataFrame(dependency_features_dict)
dependency_features_df.to_csv(LABELED_DEPENDENCY_FEATURES_FILEPATH_NO_UNK, index=None)

Found pairs:  7241
AMR Pair labels:  12147
Dependency Parser Pair labels:  11794
Not found:  4553
Pair Precision:  0.613956248940139
Pair Recall:  0.5961142668971763
Pair F1:  0.6049037216490539


In [25]:
pair_labels = create_pair_labels(amr_data)
labels_no_unk, unk_pairs, found_or_not,_ = create_labels(pair_labels,filtered_sentence_dataset, change_unk_to_mod=True)
dependency_features_dict = filtered_sentence_dataset.to_dict()
dependency_features_dict['label'] = pd.Series(labels_no_unk)
dependency_features_df = pd.DataFrame(dependency_features_dict)
dependency_features_df.to_csv(LABELED_DEPENDENCY_FEATURES_FILEPATH_NO_UNK, index=None)

Found pairs:  7241
AMR Pair labels:  12147
Dependency Parser Pair labels:  11794
Not found:  4553
Pair Precision:  0.613956248940139
Pair Recall:  0.5961142668971763
Pair F1:  0.6049037216490539


In [42]:
pair_labels

[(1, 'tanam', 'bu', 'ARG0'),
 (1, 'bu', 'ani', 'name'),
 (1, 'tanam', 'bunga', 'ARG1'),
 (2, 'tanam', 'bu', 'ARG0'),
 (2, 'bu', 'ani', 'name'),
 (2, 'tanam', 'bunga', 'ARG1'),
 (3, 'berangkat', 'om', 'ARG0'),
 (3, 'om', 'andi', 'name'),
 (3, 'berangkat', 'sekolah', 'ARG1'),
 (3, 'sekolah', 'ke', 'mod'),
 (3, 'berangkat', 'besok', 'time'),
 (4, 'berangkat', 'om', 'ARG0'),
 (4, 'om', 'andi', 'name'),
 (4, 'berangkat', 'sekolah', 'ARG1'),
 (4, 'sekolah', 'ke', 'mod'),
 (4, 'berangkat', 'besok', 'time'),
 (5, 'pergi', 'ibu', 'ARG0'),
 (5, 'ibu', 'asih', 'name'),
 (5, 'pergi', 'rumah', 'location'),
 (5, 'rumah', 'nenek', 'mod'),
 (5, 'nenek', 'yanti', 'name'),
 (5, 'pergi', 'kemarin', 'time'),
 (6, 'pergi', 'ibu', 'ARG0'),
 (6, 'ibu', 'asih', 'name'),
 (6, 'pergi', 'rumah', 'location'),
 (6, 'rumah', 'nenek', 'mod'),
 (6, 'nenek', 'yanti', 'name'),
 (6, 'pergi', 'kemarin', 'time'),
 (7, 'pergi', 'kami', 'ARG0'),
 (7, 'kami', 'keluarga', 'mod'),
 (7, 'pergi', 'tamasya', 'ARG1'),
 (7, 'pergi'

In [ ]:
filtered_sentence_dataset

In [48]:
#labels_no_unk, unk_pairs, found_or_not,_ = create_labels(pair_labels,filtered_sentence_dataset, change_unk_to_mod=True)

In [ ]:
label_found = []
for i,j in enumerate(labels_no_unk):
    temp = []
    temp.append(labels_no_unk[i])
    temp.append(found_or_not[i])
    label_found.append(temp)

In [ ]:
labels_no_unk

In [ ]:
display(dependency_features_df['label'].value_counts())

In [ ]:
dependency_features_df.tail()

### Analyze undetected pairs

In [26]:
start_idx = 350
for unk_pair in unk_pairs[start_idx:start_idx+50]:
    print(amrs[unk_pair[-1]-1].sentence)
    print("Label salah: ", unk_pair)
    print("Hasil Dependency Parser: ")
    display(dependency_features_df[dependency_features_df.sentence_id == unk_pair[-1]])
    
    print("Gold AMR:")
    print(amrs[unk_pair[-1]-1].graph)
    print()
    print('----------------------------------------------------------------------------------------------------------------')    

Roti itu busuk
Label salah:  ('roti_1', 'busuk_1', 'amod', 343)
Hasil Dependency Parser: 


,sentence_id,sequence,parent,parent_position,child,child_position,is_root,parent_ner,child_ner,parent_pos,dependency_role,child_pos,label
1075,343,3,roti_1,0,busuk_1,2,1,O,O,NN,amod,VB,mod


Gold AMR:
(b / busuk
      :ARG0 (r / roti))

----------------------------------------------------------------------------------------------------------------
Roti tersebut busuk
Label salah:  ('roti_1', 'busuk_1', 'amod', 344)
Hasil Dependency Parser: 


,sentence_id,sequence,parent,parent_position,child,child_position,is_root,parent_ner,child_ner,parent_pos,dependency_role,child_pos,label
1076,344,3,roti_1,0,busuk_1,2,1,O,O,NN,amod,VB,mod


Gold AMR:
(b / busuk
      :ARG0 (r / roti))

----------------------------------------------------------------------------------------------------------------
Roti busuk
Label salah:  ('roti_1', 'busuk_1', 'amod', 345)
Hasil Dependency Parser: 


...

Gold AMR:
(t / tampak
      :ARG0 (r / raut
            :mod (m / muka))
      :ARG1 (b / bahagia))

----------------------------------------------------------------------------------------------------------------
Nampak bahagia raut mukanya
Label salah:  ('nampak_1', 'raut_1', 'obj', 380)
Hasil Dependency Parser: 


,sentence_id,sequence,parent,parent_position,child,child_position,is_root,parent_ner,child_ner,parent_pos,dependency_role,child_pos,label
1178,380,2,nampak_1,0,bahagia_1,1,1,O,O,NN,amod,NN,mod
1179,380,3,nampak_1,0,raut_1,2,1,O,O,NN,obj,NN,ARG1
1180,380,4,raut_1,2,muka_1,3,0,O,O,NN,compound,RB,mod


Gold AMR:
(t / tampak
      :ARG0 (r / raut
            :mod (m / muka))
      :ARG1 (b / bahagia))

----------------------------------------------------------------------------------------------------------------
Kejadian kemarin membuat Rifqy tersadar
Label salah:  ('jadi_1', 'kemarin_1', 'amod', 381)
Hasil Dependency Parser: 


,sentence_id,sequence,parent,parent_position,child,child_position,is_root,parent_ner,child_ner,parent_pos,dependency_role,child_pos,label
1181,381,1,buat_1,2,jadi_1,0,1,O,O,VB,nsubj,NN,ARG0
1182,381,2,jadi_1,0,kemarin_1,1,0,O,O,NN,amod,NN,mod
1183,381,4,buat_1,2,rifqy_1,3,1,O,ORG,VB,obj,NNP,ARG1
1184,381,5,buat_1,2,sadar_1,4,1,O,O,VB,xcomp,NNP,mod


Gold AMR:
(b / buat
      :ARG0 (j / jadi)
      :ARG1 (r / rifqy)
      :mod (s / sadar)
      :time (k / kemarin))

----------------------------------------------------------------------------------------------------------------


## Test 

#### Test data

In [27]:
test_amr_data = amr_dataset['test']
test_pair_labels = create_pair_labels(test_amr_data)

In [28]:
test_labels, unk_pairs,_,_ = create_labels(test_pair_labels,filtered_test_sentence_dataset, change_unk_to_mod=True)
filtered_test_dependency_feature_dict = filtered_test_sentence_dataset.to_dict()
print(len(test_labels))
filtered_test_dependency_feature_dict['label'] = pd.Series(test_labels)
filtered_test_dependency_feature_df = pd.DataFrame(filtered_test_dependency_feature_dict)
print(len(filtered_test_dependency_feature_df))
display(filtered_test_dependency_feature_df.tail())
filtered_test_dependency_feature_df.to_csv(LABELED_TEST_DEPENDENCY_FEATURES_FILEPATH, index=None)

Found pairs:  786
AMR Pair labels:  1089
Dependency Parser Pair labels:  1084
Not found:  298
Pair Precision:  0.7250922509225092
Pair Recall:  0.721763085399449
Pair F1:  0.723423838011965
1084
1084


,sentence_id,sequence,parent,parent_position,child,child_position,is_root,parent_ner,child_ner,parent_pos,dependency_role,child_pos,label
1079,305,5,minggu_1,3,depan_1,4,0,CRD,O,NN,amod,NN,mod
1080,306,1,pilih_1,1,pasang_1,0,1,O,O,VB,nsubj:pass,NN,ARG1
1081,306,3,pilih_1,1,kakak_1,2,1,O,O,VB,obj,VB,ARG0
1082,306,4,kakak_1,2,minggu_1,3,0,O,CRD,VB,compound,NN,mod
1083,306,5,minggu_1,3,depan_1,4,0,CRD,O,NN,amod,NN,mod


In [47]:
test_labels, unk_pairs,_,_ = create_labels(test_pair_labels,filtered_test_sentence_dataset, change_unk_to_mod=True)
filtered_test_dependency_feature_dict = filtered_test_sentence_dataset.to_dict()
print(len(test_labels))
filtered_test_dependency_feature_dict['label'] = pd.Series(test_labels)
filtered_test_dependency_feature_df = pd.DataFrame(filtered_test_dependency_feature_dict)
print(len(filtered_test_dependency_feature_df))
display(filtered_test_dependency_feature_df.tail())
filtered_test_dependency_feature_df.to_csv(LABELED_TEST_DEPENDENCY_FEATURES_FILEPATH, index=None)

Found pairs:  786
AMR Pair labels:  1089
Dependency Parser Pair labels:  1084
Not found:  298
Pair Precision:  0.7250922509225092
Pair Recall:  0.721763085399449
Pair F1:  0.723423838011965
1084
1084


,sentence_id,sequence,parent,parent_position,child,child_position,is_root,parent_ner,child_ner,parent_pos,dependency_role,child_pos,label
1079,305,5,minggu_1,3,depan_1,4,0,CRD,O,NN,amod,NN,mod
1080,306,1,pilih_1,1,pasang_1,0,1,O,O,VB,nsubj:pass,NN,ARG1
1081,306,3,pilih_1,1,kakak_1,2,1,O,O,VB,obj,VB,ARG0
1082,306,4,kakak_1,2,minggu_1,3,0,O,CRD,VB,compound,NN,mod
1083,306,5,minggu_1,3,depan_1,4,0,CRD,O,NN,amod,NN,mod


### Test Berita

In [30]:
for filename in TEST_BERITA_FILE_NAMES:
    print(filename)
    berita_dependency_feature_df, amr_df, all_sentences, berita_amrs = create_dependency_parser_feature_df(TEST_BERITA_DIR+filename, ANNOTATOR_PARAMS)
    filtered_berita_dependency_features_df = filter_dataset(berita_dependency_feature_df, mode=FILTER_TYPE)
    
    # Berita pair label
    data_params = dict(
        data_dir=TEST_BERITA_DIR,
        train_data="{}.features".format(filename),
        test_data="{}.features".format(filename),
        dev_data="{}.features".format(filename),
        data_type="AMR"
    )
    
    berita_amr_data = dataset_from_params(data_params)['test']
    berita_pair_labels = create_pair_labels(berita_amr_data)

    berita_labels, unk_pairs,_,_ = create_labels(berita_pair_labels, filtered_berita_dependency_features_df, change_unk_to_mod=True)
    filtered_berita_dependency_features_df.insert(len(filtered_berita_dependency_features_df.columns), "label", berita_labels)
    
    topic = filename.split('.')[0]
    filtered_berita_dependency_features_df.to_csv(TEST_BERITA_DIR+"labeled_df_"+topic+'.csv', index=None)

b-salah-darat.txt


0it [00:00, ?it/s][2021-05-11 21:46:09,403 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/b-salah-darat.txt.features
32it [00:00, 434.95it/s]
0it [00:00, ?it/s][2021-05-11 21:46:09,478 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/b-salah-darat.txt.features
32it [00:00, 1104.43it/s]
0it [00:00, ?it/s][2021-05-11 21:46:09,507 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/b-salah-darat.txt.features
32it [00:00, 689.80it/s]

Building train datasets ...
False
Building dev datasets ...
False
Building test datasets ...
False


Found pairs:  292
AMR Pair labels:  461
Dependency Parser Pair labels:  442
Not found:  150
Pair Precision:  0.6606334841628959
Pair Recall:  0.6334056399132321
Pair F1:  0.6467331118493909
c-gedung-roboh.txt


0it [00:00, ?it/s][2021-05-11 21:46:21,014 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/c-gedung-roboh.txt.features
29it [00:00, 587.35it/s]
0it [00:00, ?it/s][2021-05-11 21:46:21,065 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/c-gedung-roboh.txt.features
29it [00:00, 824.43it/s]
0it [00:00, ?it/s][2021-05-11 21:46:21,103 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/c-gedung-roboh.txt.features
29it [00:00, 829.95it/s]

...

0it [00:00, ?it/s][2021-05-11 21:48:51,704 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/g-gempa-dieng.txt.features
19it [00:00, 211.75it/s]
0it [00:00, ?it/s][2021-05-11 21:48:51,811 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/g-gempa-dieng.txt.features


Building train datasets ...
False
Building dev datasets ...
False


19it [00:00, 244.25it/s]
0it [00:00, ?it/s][2021-05-11 21:48:51,902 INFO] Reading instances from lines in file at: C:/Users/Asus/JupyterNotebook/Tesis/amr_parser-master(trainUlangAdylan)/data_amany/data/raw/test/g-gempa-dieng.txt.features
19it [00:00, 176.40it/s]

Building test datasets ...
False


Found pairs:  148
AMR Pair labels:  267
Dependency Parser Pair labels:  265
Not found:  117
Pair Precision:  0.5584905660377358
Pair Recall:  0.5543071161048689
Pair F1:  0.5563909774436091


In [ ]:
test_data = pd.read_csv(TEST_BERITA_DIR+"labeled_df_b-salah-darat.csv")
test_data['child_pos'].value_counts()